# Analysis of Reviews Containing Identified Stress Mentions

**Purpose**: This notebook delves into the reviews that have been flagged for containing one or more of the stress mentions identified in the `medical-entity-analysis.ipynb` notebook. The aim is to understand the context and prevalence of these mentions within the broader dataset.

**Description**: 
- **Filtering Process**: The initial step involves filtering the reviews to retain only those that match the identified stress mentions.
- **Observational Analysis**: A basic observational analysis is conducted on the filtered dataset to understand patterns, frequencies, and other key metrics.
- **Stress Score Extraction**: Adhering to the methodology described in the literature, a stress score will be computed for each company based on the propotion of tis posts related to stress. This score provides a quantitative measure of the stress level associated with each company.
- **Comparative Analysis**: The notebook also compares key statistics between the entire dataset and the filtered subset. This comparison provides insights into how the presence of stress mentions affects the overall tone and content of reviews.

In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Set the Pandas display option to show the entire content of each cell without truncation
pd.set_option('display.max_colwidth', None)

In [3]:
GREEN = "\033[92m"  # ANSI escape code for green text color
ENDC = "\033[0m"    # ANSI escape code to reset text color to default

## 1. Data Loading

In [4]:
# Specify the path to the folder containing the data (relative)
path = os.path.join("..", "data")

# File containing the stress entities
txt_fpath = os.path.join(path, "extracted", "stress_entities.txt")

# File containing glassdoor's reviews
reviews_fpath = os.path.join(path, "reviews.csv")

# File containing glassdoor companies
companies_fpath = os.path.join(path, "crawler", "companies.json")

In [5]:
# Load the TXT file into a Python List object
stress_entities = []

with open(txt_fpath, "r") as file:
    text = file.read()

    text = text.replace("'", "")
    text = text.replace("[", "").replace("]", "")
    
    stress_entities = text.split(",")

In [6]:
# Load the reviews CSV file into a Pandas DataFrame object
reviews = pd.read_csv(reviews_fpath)

In [7]:
# Load the companies CSV file into a Pandas DataFrame object
companies = pd.read_json(companies_fpath)

## 2. Preview of Stress Mentioned Reviews

The extracted list of stress entities will now be used to sample some reviews that contain the related mentions

In [8]:
def find_reviews_with_pattern(pattern, case=False, n=10):
    # Filter reviews to find rows where the summary collumn contains the desired pattern
    matching_reviews = reviews[reviews["summary"].str.contains(pattern, case=case)]
    matching_reviews = matching_reviews["summary"]
    matching_reviews.reset_index(drop=True, inplace=True)
    
    count = matching_reviews.shape[0]
    matching_reviews = matching_reviews.head(n)

    return (matching_reviews, count)

In [9]:
for entity in stress_entities:
    matching_reviews, count = find_reviews_with_pattern(entity)

    print(f"Total matching reviews for {GREEN}{entity}{ENDC}: {count}")
    print(matching_reviews, "\n")

Total matching reviews for stress: 25489
0                                                                                      Stressful & Frustrating
1                                                                                                    Stressful
2                                                                                                   Stressfull
3                                                                  My experience consists of being stressed...
4    Fun people, chaotic day-to-day, no work-life balance, poor information sharing, high stress, no training.
5                                                                                  Stressful at times but fine
6                                                                      Disorganized, stressful and incompetent
7                                                                                         very stress full job
8                                                                    Hi

## 3. Filtering Reviews

Now we will generate a new DataFrame that retains only the rows which their review contains at least one of the identified stress mentions.

In [10]:
mask = reviews["summary"].str.contains("|".join(stress_entities), case=False)

In [11]:
filtered_reviews = reviews[mask].copy()
filtered_reviews.reset_index(drop=True, inplace=True)

In [12]:
filtered_reviews["summary"].sample(10)

5719                               Stressful at times but overall good experience
38308                                 Stressful and not advisable company to join
5553                                                                    Stressful
39720                                                               Growing Pains
2850                                                 High Stress Work Environment
4147     Cheap place to work, drastic year over year cuts, overworked, underpaid.
25018                                                 Hostile, Chaotic, Stressful
48329                                                                 Stress-free
32997                                                       Way too much pressure
33430                                   Great Opportunities, but Very High Stress
Name: summary, dtype: object

In [13]:
# How many reviews are we left with?
count = filtered_reviews.shape[0]
print("Reviews left:", count)

Reviews left: 48631


Let us also add a new boolean column to the existing DataFrame indicating whether a stress metnion is present in the summary column

In [14]:
reviews["has_stress"] = mask

In [15]:
reviews[reviews["has_stress"] == 1][["has_stress", "summary"]].sample(5)

,has_stress,summary
8841711,True,Stress free work environment
3876938,True,New divisions and launches have growing pains like no other
304867,True,Huge pressure
2521004,True,Not worth the stress if you aren't planning on being a closer every night
1945029,True,"Stressful, but good pay."


## 4. Stress Score Extraction

### 4.1. Associating Stress Types with Companies
Following the literture's methodology, we will compute the stress score for each company based on the proportion of its posts related to stress. This involves calculating the average review rating and stress score for each company, followed by z-scoring these values for comparability.

Since we want to find how COVID-19 affected the reviews and the overall stress of the employees we will compute the rating and stress before and after covid.

In [16]:
# Cut-off points of COVID-19
before_date = "2020-01-30"
after_date = "2021-08-23"

In [17]:
reviews["review_date_time"] = pd.to_datetime(reviews["review_date_time"])

In [18]:
# Acquire the reviews submited before COVID-19 
reviews_before = reviews[reviews["review_date_time"] <= before_date]

# Acquire the reviews submited during COVID-19
reviews_during = reviews[
    (reviews["review_date_time"] > before_date) &
    (reviews["review_date_time"] <= after_date)]

# Acquire the reviews submited after COVID-19
reviews_after = reviews[reviews["review_date_time"] > after_date]

In [19]:
print(reviews_before.shape[0], "reviews before covid", f"({before_date} <=)")
print(reviews_during.shape[0], "reviews during covid", f"({before_date} > AND {after_date} <=)")
print(reviews_after.shape[0], "reviews after covid", f"({after_date} >)")

4014699 reviews before covid (2020-01-30 <=)
1985713 reviews during covid (2020-01-30 > AND 2021-08-23 <=)
2953072 reviews after covid (2021-08-23 >)


In [20]:
def compute_average_rating_per_company(df):
    # Group by "company_id" and calculate mean of "ratingOverall"
    result = df[["rating_overall", "company_id"]].groupby(by="company_id").mean().reset_index()
    result.columns = ["company_id", "average_rating"]

    # Compute z-score for rating
    result["zrating"] = (result["average_rating"] - result["average_rating"].mean()) / result["average_rating"].std()
    return result

In [21]:
def compute_stress_score_per_company(df):
    # Reviews per company that had stress mentions
    true_counts = df[["company_id", "has_stress"]].groupby(by="company_id").sum()
    
    # Total number of reviews per company
    total_counts = df[["company_id", "has_stress"]].groupby(by="company_id").count()

    # Compute their ratio / stress score
    result = true_counts / total_counts

    result.reset_index(inplace=True)
    result.columns = ["company_id", "stress"]

    # Drop companies with 0 stress
    result = result[result["stress"] != 0]
    
    # Compute z-score for stress
    result["zstress"] = (result["stress"] - result["stress"].mean()) / result["stress"].std()

    return result

In [22]:
def calculate_association(df):
    R = np.sqrt(np.power(df["zrating"], 2) + np.power(df["zstress"], 2))

    alpha = np.arccos(np.abs(df["zrating"]) / R)
    beta = np.arccos(np.abs(df["zstress"]) / R)
    gamma = np.maximum((alpha - np.pi / 4), (beta - np.pi / 4))

    return pd.Series(R / (gamma + np.pi))

In [23]:
avg_rating_before = compute_average_rating_per_company(reviews_before)
stress_before = compute_stress_score_per_company(reviews_before)

results_before = pd.merge(left=avg_rating_before, right=stress_before, on="company_id")
results_before = pd.merge(left=companies, right=results_before, left_on="_id", right_on="company_id")

results_before["association"] = calculate_association(results_before)

del results_before["_id"]

In [24]:
avg_rating_during = compute_average_rating_per_company(reviews_during)
stress_during = compute_stress_score_per_company(reviews_during)

results_during = pd.merge(left=avg_rating_during, right=stress_during, on="company_id")
results_during = pd.merge(left=companies, right=results_during, left_on="_id", right_on="company_id")

results_during["association"] = calculate_association(results_during)

del results_during["_id"]

In [25]:
avg_rating_after = compute_average_rating_per_company(reviews_after)
stress_after = compute_stress_score_per_company(reviews_after)

results_after = pd.merge(left=avg_rating_after, right=stress_after, on="company_id")
results_after = pd.merge(left=companies, right=results_after, left_on="_id", right_on="company_id")

results_after["association"] = calculate_association(results_after)

del results_after["_id"]

Plot Extracted Quadrants

In [26]:
quadrants = {
    "Q1": {"name": "Positive Stress", "color": "orange", "position": (3, 3)},  # Top-right quadrant
    "Q2": {"name": "Negative Stress", "color": "red", "position": (-3, 3)},   # Top-left quadrant
    "Q3": {"name": "Passive", "color": "black", "position": (-3, -3)},        # Bottom-left quadrant
    "Q4": {"name": "Low Stress", "color": "green", "position": (3, -3)}       # Bottom-right quadrant
}

In [27]:
def plot_quadrants(df, title):
    # Determine the quadrant for each point
    df["quadrant"] = df.apply(lambda row: "Q1" if row["zrating"] >= 0 and row["zstress"] >= 0 else 
                              ("Q2" if row["zrating"] < 0 and row["zstress"] >= 0 else 
                              ("Q3" if row["zrating"] < 0 and row["zstress"] < 0 else "Q4")), axis=1)
    
    # Create the base figure
    fig = go.Figure()
    
    # Add scatter points for each quadrant
    for quad, details in quadrants.items():
        df_quad = df[df["quadrant"] == quad]
        fig.add_trace(go.Scatter(x=df_quad["zrating"], y=df_quad["zstress"], 
                                 mode="markers", 
                                 marker=dict(color=details["color"]),
                                 name=details["name"], 
                                 hovertext=df_quad["company name"]))
        
        # Add annotation (text) to indicate the number of companies in the quadrant
        fig.add_annotation(
            text=str(len(df_quad)),
            x=details["position"][0], 
            y=details["position"][1],
            showarrow=False,
            font=dict(
                size=15,
                color=details["color"]
            ),
            bgcolor="white",
            borderpad=4
        )
    
    # Add vertical line at x=0
    fig.add_shape(type="line", line=dict(dash="dash"), x0=0, x1=0, y0=-3, y1=3)
    
    # Add horizontal line at y=0
    fig.add_shape(type="line", line=dict(dash="dash"), y0=0, y1=0, x0=-3, x1=3)
    
    # Update layout, titles, and axis ranges
    fig.update_layout(title=f"Stress Quadrants: {title}", 
                      xaxis_title="zrating", 
                      yaxis_title="zstress",
                      legend_title="Quadrants",
                      height=800)

    # Display the plot
    fig.show()

In [28]:
plot_quadrants(df=results_before, title=f"Before COVID-19 ({before_date} <)")

In [29]:
plot_quadrants(results_during, title=f"During COVID-19 ({before_date} - {after_date})")

In [30]:
plot_quadrants(results_after, title=f"After COVID-19 ({after_date} >)")

Plots but only for companies which were in every period

How many companies do we have per period?

In [31]:
print(results_before.shape[0], "companies before covid", f"({before_date} <=)")
print(results_during.shape[0], "companies during covid", f"({before_date} > AND {after_date} <=)")
print(results_after.shape[0], "companies after covid", f"({after_date} >)")

5248 companies before covid (2020-01-30 <=)
3359 companies during covid (2020-01-30 > AND 2021-08-23 <=)
4262 companies after covid (2021-08-23 >)


Lets merge these 3 dataframes into 1

In [32]:
# Add a suffix to the columns so we know which dataframe they come from
temp_before = results_before.add_suffix("_before")
temp_during = results_during.add_suffix("_during")
temp_after = results_after.add_suffix("_after")

# Reset the "company_id" column name so it can be used for merging
temp_before.rename(columns={"company_id_before": "company_id"}, inplace=True)
temp_during.rename(columns={"company_id_during": "company_id"}, inplace=True)
temp_after.rename(columns={"company_id_after": "company_id"}, inplace=True)

# Perform the first merge
merged_df = pd.merge(temp_before, temp_during, on="company_id", how="outer")

# Perform the second merge
results_df = pd.merge(merged_df, temp_after, on="company_id", how="outer")

In [33]:
# Drop the redundant company name columns
results_df.drop(columns=["company name_during", "company name_after"], inplace=True)

# Rename the remaining company name column
results_df.rename(columns={"company name_before": "company_name"}, inplace=True)

In [34]:
results_df.shape

(7057, 20)

In [35]:
results_df.sample(5).T

,775,3727,1458,3336,2395
company_name,Dropbox,County of San Mateo,Atlantic Health System,Donnelley Financial Solutions (DFIN),Nemours Children's Health
company_id,415350,270293,17694,1431223,18241
average_rating_before,4.262673,3.429907,3.598746,2.931624,3.604061
zrating_before,1.530857,-0.210114,0.14286,-1.251817,0.153971
stress_before,0.013825,0.028037,0.003135,0.008547,0.005076
zstress_before,0.298753,1.720342,-0.77051,-0.22916,-0.576329
association_before,0.417683,0.455431,0.209325,0.339734,0.162726
quadrant_before,Q1,Q2,Q4,Q3,Q4
average_rating_during,3.902834,NaN,3.741573,NaN,3.760417
zrating_during,0.273384,NaN,-0.121727,NaN,-0.075557


In [36]:
companies_in_all_periods = results_df.dropna()["company_id"]
companies_in_all_periods = companies_in_all_periods.reset_index()

In [37]:
# Merge with results_before, results_during, and results_after on "company_id"
merged_df_before = pd.merge(companies_in_all_periods, results_before, on="company_id", how="inner")
merged_df_during = pd.merge(companies_in_all_periods, results_during, on="company_id", how="inner")
merged_df_after = pd.merge(companies_in_all_periods, results_after, on="company_id", how="inner")

In [38]:
plot_quadrants(df=merged_df_before, title=f"Before COVID-19 ({before_date} <)")

In [39]:
plot_quadrants(merged_df_during, title=f"During COVID-19 ({before_date} - {after_date})")

In [40]:
plot_quadrants(merged_df_after, title=f"After COVID-19 ({after_date} >)")

In [41]:
results_df.to_csv(os.path.join(path, "extracted", "stress_per_company.csv"))

### 4.2. Computing Stress Scores Over the Years

In [42]:
# Compute the scores for the whole data (without cutoff point)
avg_rating = compute_average_rating_per_company(reviews)
stress = compute_stress_score_per_company(reviews)

results = pd.merge(left=avg_rating, right=stress, on="company_id")
results = pd.merge(left=companies, right=results, left_on="_id", right_on="company_id")

results["association"] = calculate_association(results)

del results["_id"]

In [43]:
plot_quadrants(results, "All Years")

In [44]:
def calculate_weight(df):
    temp = df.groupby(by="company_id").size()
    temp = temp.reset_index(name="count")

    temp["weight"] = temp["count"] / df.shape[0]

    return temp

In [45]:
temp = pd.merge(
    reviews,
    results[["company_id", "quadrant"]],
    on="company_id"
)

In [46]:
temp["year"] = temp["review_date_time"].dt.year

In [47]:
grouped = temp.groupby(by=["year", "quadrant"])

In [48]:
stress_over_years = {}

for (year, quadrant), group in grouped:
    
    # Compute the scores for the whole data (without cutoff points)
    avg_rating = compute_average_rating_per_company(group)
    stress = compute_stress_score_per_company(group)

    results = pd.merge(left=avg_rating, right=stress, on="company_id")
    results = pd.merge(left=companies, right=results, left_on="_id", right_on="company_id")

    results["association"] = calculate_association(results)

    weights = calculate_weight(group)
    results = pd.merge(left=results, right=weights, on="company_id")

    del results["_id"]
    del results["count"]

    # Create a unique key for the year and quadrant combination
    year_quadrant_key = (year, quadrant)
    
    stress_over_years[year_quadrant_key] = np.sum(results["association"] * results["weight"])

In [49]:
# Convert your dictionary into a DataFrame
data = []
for (year, quadrant), value in stress_over_years.items():
    data.append({'year': year, 'quadrant': quadrant, 'value': value})

df = pd.DataFrame(data)

In [50]:
# Now create the line plot
fig = px.line(
    df, 
    x="year", 
    y="value", 
    color="quadrant", 
    title='Stress Over Years per Quadrant',
    labels={"value": "Stress"},
    line_shape="linear"
)

# Update trace names and colors based on the quadrants dictionary
for trace, quadrant_key in zip(fig.data, quadrants.keys()):
    quadrant_info = quadrants[quadrant_key]
    trace.name = quadrant_info['name']
    trace.line.color = quadrant_info['color']

fig.update_layout(legend_title_text="Quadrants")
fig.update_xaxes(title_text="Year")

# Show the plot
fig.show()

In [51]:
# Compute the average value for each year
average_values = df.groupby('year')['value'].mean().reset_index()

# Create a new DataFrame with these average values
average_df = pd.DataFrame(average_values)

# Now create the line plot for the average values
fig = px.line(
    average_df, 
    x="year", 
    y="value",
    title="Average Stress Over Years",
    labels={"value": "Average Stress"},
    line_shape="linear"
)

fig.update_xaxes(title_text="Year")

# Show the plot
fig.show()